In [1]:
!pip install easyfsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00


In [2]:
print("hi")
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import random
from statistics import mean

import torch
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split #to do
from sklearn.preprocessing import OrdinalEncoder
#import seaborn as sns #vis
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torchvision
from torch.optim import Adam
from tqdm import tqdm
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
random_seed = 2023
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
batch_size = 2**9 #2**15
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
df.head()
x=df.drop("Class",axis=1)
y=df["Class"]
y[y==1].count()
X_train, X_test, y_train, y_test = train_test_split(x, y,
    test_size=0.4, shuffle = True, random_state = 8)

# Use the same function above for the validation set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, 
    test_size=0.5, random_state= 8) # 0.25 x 0.8 = 0.2
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y val shape: {}".format(y_val.shape))
print(y_train[y_train==1].count())
print(y_val[y_val==1].count())
print(y_test[y_test==1].count())
x = X_train.values  # Input values.
y = y_train.values    # Output values(species categories)
xv=X_val.values
yv=y_val.values
x_train = torch.FloatTensor(x)
y_train = torch.tensor(y)
x_val= torch.FloatTensor(xv)
y_val = torch.tensor(yv)
x_train_reshaped=x_train.reshape(170884,1,1,30).expand(-1,3,-1,-1)
x_val_reshaped=x_val.reshape(56962,1,1,30).expand(-1,3,-1,-1)
x_val_reshaped.shape


hi
X_train shape: (170884, 30)
y_train shape: (170884,)
X_test shape: (56961, 30)
y_test shape: (56961,)
X_val shape: (56962, 30)
y val shape: (56962,)
278
108
106


torch.Size([56962, 3, 1, 30])

In [3]:
n_way = 2
n_shot = 10
n_query = 10
n_tasks_per_epoch = 100
n_validation_tasks = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_workers = 2

In [4]:
train_set = data_utils.TensorDataset(x_train_reshaped, y_train)
train_set.get_labels = lambda:  [ y.item() for y in y_train]
val_set = data_utils.TensorDataset(x_val_reshaped, y_val)
val_set.get_labels = lambda:  [ y.item() for y in y_val]

In [5]:
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

In [6]:
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    #num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    #num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

In [7]:
from easyfsl.methods import RelationNetworks
from torchvision.models import  resnet18,ResNet18_Weights
backbone=resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
backbone = torch.nn.Sequential(*(list(backbone.children())[:-1]))
#backbone.eval()
few_shot_classifier = RelationNetworks(backbone).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
from torch.optim import SGD, Optimizer,Adam
from torch.optim.lr_scheduler import MultiStepLR
#from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = .001
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

#tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [9]:
def training_epoch(
    model, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(support_images.to(device), support_labels.to(device))
            #model.process_query_set(query_images.to(device), query_labels.to(device))
            #print(torch.isnan(query_images).sum())#torch.isnan(torch.tensor([1, float('nan'), 2])).sum()
            classification_scores = model(query_images.to(device))
            #print("classification_scores ",classification_scores)
            print(query_labels)
            #that string is what fix that erorr
            """for i in range(len(query_labels)):
                if query_labels[i] !=0:
                    query_labels[i]=1"""
            #print("classification_scores ",classification_scores)
            loss = LOSS_FUNCTION(classification_scores, query_labels.to(device))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [10]:
best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0

In [11]:
from easyfsl.methods.utils import evaluate

print("I'm not dead")
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=device, tqdm_prefix="Validation"
    )
    print(validation_accuracy)
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")
        torch.save(few_shot_classifier.state_dict(),"checkpoint1.pth")
    #tb_writer.add_scalar("Train/loss", average_loss, epoch)
    #tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()
    

I'm not dead
Epoch 0


Training:   0%|          | 0/100 [00:00<?, ?it/s]

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


Training:   0%|          | 0/100 [00:00<?, ?it/s]


IndexError: Target 7 is out of bounds.